# Brains4Buildings interactive inverse grey-box analysis pipeline

This Jupyter Labs notebook can be used to interactively test the Brains4Buildings inverse grey-box analysis pipeline.
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!

This file be downloaded from the [twomes-dataset-windesheim-brains4buildings2022 repository](https://edu.nl/cvwtj):
- `raw_properties/b4b_raw_properties.parquet`; 
- `metadata/b4b-room-metadata.zip`

## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

from tqdm.notebook import tqdm
from gekko import GEKKO

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

base_url = 'https://github.com/energietransitie/twomes-dataset-windesheim-brains4buildings2022/raw/main/'

#location: Google Maps location of Hogeschool Windesheim in Zwolle
lat, lon = 52.499255, 6.0765167
timezone_ids = 'Europe/Amsterdam'

from preprocessor import Preprocessor
from inversegreyboxmodel import Learner
from plotter import Plot

%load_ext autoreload
%matplotlib inline
%matplotlib widget

import logging
logging.basicConfig(level=logging.ERROR, 
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    filename='log_b4b.txt',
                   )

### Load Measured Data from parquet file

In [ ]:
%%time
# Prerequisite: for this example to work, you need to have the b4b_raw_properties.parquet, located in the ../data/ folder.
# One way to get this is to run B4BExtractionBackup.ipynb first, but then you have to run this code on the energietransitiewindesheim.nl server

file_prop = 'raw_properties/b4b_raw_properties.parquet'
df_prop = pd.read_parquet(base_url + file_prop, engine='pyarrow')

# df_prop = pd.read_parquet('https://github.com/energietransitie/twomes-dataset-windesheim-brains4buildings2022/raw/main/raw_properties/b4b_raw_properties.parquet',
#                           engine='pyarrow')

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFramews are properly sorted
if not df_prop.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prop = df_prop.sort_index()  

In [ ]:
df_prop.index.unique(level='id').values

In [ ]:
df_prop.index.unique(level='source').values

In [ ]:
df_prop

In [ ]:
df_prop.info()

## Preprocessing including merging weather data

In [ ]:
df_prep = Preprocessor.preprocess_room_data(df_prop, lat, lon, timezone_ids)

In [ ]:
df_prep.index.unique(level='id').values

In [ ]:
df_prep.info()

In [ ]:
df_prep.describe()

In [ ]:
df_prep

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$',
    'm_s_1': r'$m/s$',
    'W_m_2' : r'$W/m^2$'
}

In [ ]:
# visualize input data
df_plot = df_prep

In [ ]:
df_plot.info()

In [ ]:
#Plot all properties from all sources for all ids
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

## Learn parameters using inverse grey-box analysis

Most of the heavy lifting is done by the `learn_room_parameters()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
%%time 
%autoreload 2

# learn the model parameters for only a subset of the room ids and write results to a dataframe

# read room metadata

file_metadata = 'metadata/b4b-room-metadata.zip'
df_room_metadata = pd.read_csv(base_url + file_metadata, usecols=['id', 'room__m3', 'vent_max__m3_h_1']).set_index(['id'])

# filename = '../data/b4b-room-metadata.zip'
# df_room_metadata = pd.read_csv(filename, usecols=['id', 'room__m3', 'vent_max__m3_h_1']).set_index(['id'])

hints = {
    'A_inf__m2' : (10 / 1e4) # 10 [cm^2] 
}
    
#select whether to learn a time-varying parameters as well (NB you can set 0 or 1 of these to true, both not both
learn = [
    'A_inf__m2',
    'valve_frac__0',
    # 'occupancy__p',
]

#select column names
property_sources = {
    'co2__ppm' : 'CO2-meter-SCD4x_co2__ppm',           # options: ['bms_co2__ppm', 'CO2-meter-SCD4x_co2__ppm']
    'occupancy__p': 'CO2-meter-SCD4x_occupancy__p',    # options: ['CO2-meter-SCD4x_occupancy__p', 'xovis_occupancy__p']
    'valve_frac__0': 'bms_valve_frac__0'
}

df_results_per_period, df_results = Learner.learn_room_parameters(df_prep,
                                                                  property_sources = property_sources, 
                                                                  df_metadata = df_room_metadata,
                                                                  hints = hints,
                                                                  learn = learn,
                                                                  learn_period__d = 3, 
                                                                  learn_change_interval__min = 30,
                                                                  ev_type=2
                                                                 )


### Result per learning period

In [ ]:
#write df_results_per_period to zipped CSV file for analysis in other programs
if any(df_results.columns.str.startswith('model_')): 
    roomtype='virtual'
else:
    roomtype='real'
    
if (('valve_frac__0' in learn) or ('occupancy__p'in learn)):
    if 'valve_frac__0' in learn:
        resulttype = 'valve_frac__0'
    else:
        resulttype = 'occupancy__p'
else:
    resulttype= 'A_inf__m2'

df_results_per_period.to_csv(f'results_{resulttype}_per_period_{roomtype}_rooms.zip',
                             encoding='utf-8',
                             compression= dict(method='zip',
                                               archive_name=f'results_{resulttype}_per_period_{roomtype}_rooms.csv'),
                             date_format='%Y-%m-%dT%H:%M:%S%z')

In [ ]:
df_results_per_period

In [ ]:
# box plot for 'learned_A_inf__cm2' per room 

Plot.learned_parameters_boxplot_b4b(df_results_per_period, parameter='learned_A_inf__cm2')

In [ ]:
# show essential statistics for the learned values
df_stats = df_results_per_period.describe().filter(regex='^actual_|^learned_')
df_stats.loc[df_stats.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
# show essential statistics for the errors
df_stats = df_results_per_period.describe().filter(regex='^mae_|^rmse')
df_stats.loc[df_stats.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
# show essential statistics for the error values, per id
df_stats = df_results_per_period.groupby('id').describe().stack().filter(regex='^mae_|^rmse')
df_stats.loc[df_stats.index.get_level_values(1).isin(['mean', 'std', 'min', 'max'])]

### Result Visualization

In [ ]:
#Plot all properties from all sources for all ids
input_props = list(property_sources.values())
learned_props = input_props + ['sim_co2__ppm']
learned_props_frac = input_props + ['sim_co2__ppm', 'learned_valve_frac__0']
learned_props_occupancy = input_props + ['sim_co2__ppm', 'learned_occupancy__p']

In [ ]:
df_results

In [ ]:
# Scatter plot of two features (for example sim_co2__ppm vs learned_valve_frac__0)

Plot.learned_parameters_scatterplot(df=df_results, parameters=['sim_co2__ppm', 'bms_valve_frac__0'])

In [ ]:
# select properties to visualise
if (('valve_frac__0' in learn) or ('occupancy__p'in learn)):
    if 'valve_frac__0' in learn:
        props = learned_props_frac
    else:
        props = learned_props_occupancy
else:
    props = learned_props
        
# df_plot = df_prep[props]

df_plot = df_results[props]

In [ ]:
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

In [ ]:
df_plot

In [ ]:
df_plot.info()